<a href="https://colab.research.google.com/github/taddbackus/capstone/blob/main/Data_Cleanup_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
https://geemap.org/notebooks/114_dynamic_world/


# Imports and Setup

In [1]:
!pip -q install --upgrade folium
!pip -q install geopandas
!pip -q install geojson
!pip -q install eeconvert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 119.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 16.5 MB/s eta 0:00:00


In [2]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

In [3]:
pip install eemont

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eemont: filename=eemont-0.3.6-py3-none-any.whl size=135742 sha256=2a9076353f72c23c83f8c28ef805211ba51207c6280f943d68d986e0c0cb222a
  Stored in directory: /root/.cache/pip/wheels/c2/79/2c/4fed17c3d3b466bbf4fe5872eec11f189147043b01152a4f75
  Created wheel for ee_extra: filename=ee_extra-0.0.15-py3-none-any.whl size=236755 sha256=f686851b364d3afed532e33aa256ae3a212341a7abe09eaea899ed23b0521e1f
  Stored in directory: /root/.cache/pip/wheels/29/96/0e/4e36b0dfd85e16867723df739294c0aa45a65b191adac4d959
Successfully built eemont ee_extra


In [4]:
# Standard imports
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospatial processing packages
import geopandas as gpd
import geojson

import shapely
from shapely.geometry import box

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import folium
import eemont

import geopandas as gpd
from shapely.geometry import shape

### Mount Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [36]:
#join in overall info
# Read the GeoJSON file
mines = gpd.read_file('/content/drive/MyDrive/OSMRE_GEOMINE_GEOJSON.geojson')
mines = pd.DataFrame(mines)
mines = mines[['company',
               'coalmine_op_status',
               'mine_name',
               'permit_id',
               'state_tribe_mine_id',
               'national_id',
               'coal_bed_names',
               'inspectable_unit_status',
                'post_smcra',
               'calculated_area',
               'reported_area',
               'permit_application_type',
               'permit_application_date',
               'permit_approval_date',
               'edit_date',
               'area_mine',
               'contour',
               'mountaintop',
               'steep_slope',
               'highwall',
               'auger',
               'contact',
              'information_link']]

In [37]:
mines.columns

Index(['company', 'coalmine_op_status', 'mine_name', 'permit_id',
       'state_tribe_mine_id', 'national_id', 'coal_bed_names',
       'inspectable_unit_status', 'post_smcra', 'calculated_area',
       'reported_area', 'permit_application_type', 'permit_application_date',
       'permit_approval_date', 'edit_date', 'area_mine', 'contour',
       'mountaintop', 'steep_slope', 'highwall', 'auger', 'contact',
       'information_link'],
      dtype='object')

In [38]:
mines.head()

,company,coalmine_op_status,mine_name,permit_id,state_tribe_mine_id,national_id,coal_bed_names,inspectable_unit_status,post_smcra,calculated_area,...,permit_approval_date,edit_date,area_mine,contour,mountaintop,steep_slope,highwall,auger,contact,information_link
0,HIGGINS COAL CO INC,7,MINE #50,2404,NaN,TN2404,NaN,NaN,1.0,2.220938,...,1985-06-11,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/
1,MIDNIGHT COAL CO,7,MINE #1,2405,NaN,TN2405,NaN,NaN,1.0,2.752046,...,1985-08-29,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/
2,META ELKHORN MINING CO INC,7,AREA 11,2756,NaN,TN2756,NaN,NaN,1.0,270.299198,...,1988-11-08,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/
3,JORDAN BROTHERS COAL CO INC,7,TIPPLE #1,2497,NaN,TN2497,NaN,NaN,1.0,3.507990,...,1991-09-09,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/
4,DOUBLE Q INC,7,ANGELA #1,2284023,NaN,TN2284023,NaN,NaN,1.0,13.751147,...,1984-04-29,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/


In [101]:
import pandas as pd
import os

directory_path = '/content/drive/MyDrive/Mines_Data/'
directory_files = os.listdir(directory_path)

df = pd.DataFrame()
z=0
for i in directory_files:
    df_file = pd.read_csv(os.path.join(directory_path, i), encoding = 'ISO-8859-1',low_memory=False)
    df_file['start_date'] = directory_files[z]
    df_file['start_date'] = df_file['start_date'].str.slice(3, 7)
    df_file['end_date'] = df_file['start_date']+'-07-31'
    df_file['start_date'] = df_file['start_date']+'-07-01'
    df_file['start_date'] = pd.to_datetime(df_file['start_date'])
    df_file['end_date'] = pd.to_datetime(df_file['end_date'])

    df_file['State'] = directory_files[z]
    df_file['State'] = df_file['State'].str.slice(0, 2)

    df = pd.concat([df, df_file])

    z+=1


In [ ]:
#mapping demo in python https://colab.research.google.com/github/QuantEcon/quantecon-notebooks-datascience/blob/master/applications/maps.ipynb#scrollTo=SI5i7rboSGFL&uniqifier=2
# Read the downloaded file

0	#419bdf	water
1	#397d49	trees
2	#88b053	grass
3	#7a87c6	flooded_vegetation
4	#e49635	crops
5	#dfc35a	shrub_and_scrub
6	#c4281b	built
7	#a59b8f	bare
8	#b39fe1	snow_and_ice


label0 = df[['orig_perm_id', 'label','mine_id']][df['label']==0.0].groupby(['orig_perm_id', 'label']).count()
label0 = label0.rename(columns={"mine_id": "water"})
label1 = df[['orig_perm_id', 'label','mine_id']][df['label']==1.0].groupby(['orig_perm_id', 'label']).count()
label1 = label1.rename(columns={"mine_id": "trees"})
label2 = df[['orig_perm_id', 'label','mine_id']][df['label']==2.0].groupby(['orig_perm_id', 'label']).count()
label2 = label2.rename(columns={"mine_id": "grass"})
label3 = df[['orig_perm_id', 'label','mine_id']][df['label']==3.0].groupby(['orig_perm_id', 'label']).count()
label3 = label3.rename(columns={"mine_id": "flooded_vegetation"})
label4 = df[['orig_perm_id', 'label','mine_id']][df['label']==4.0].groupby(['orig_perm_id', 'label']).count()
label4 = label4.rename(columns={"mine_id": "crops"})
label5 = df[['orig_perm_id', 'label','mine_id']][df['label']==5.0].groupby(['orig_perm_id', 'label']).count()
label5 = label5.rename(columns={"mine_id": "shrub_and_scrub"})
label6 = df[['orig_perm_id', 'label','mine_id']][df['label']==6.0].groupby(['orig_perm_id', 'label']).count()
label6 = label6.rename(columns={"mine_id": "built"})
label7 = df[['orig_perm_id', 'label','mine_id']][df['label']==7.0].groupby(['orig_perm_id', 'label']).count()
label7 = label7.rename(columns={"mine_id": "bare"})
label8 = df[['orig_perm_id', 'label','mine_id']][df['label']==8.0].groupby(['orig_perm_id', 'label']).count()
label8 = label8.rename(columns={"mine_id": "snow_and_ice"})

df = pd.merge(df, df[['orig_perm_id', 'start_date']].groupby('orig_perm_id').count(), on='orig_perm_id', how='left')
df = df.rename(columns={"start_date_y": "Observations", "start_date_x": "Date"})

df = pd.merge(df[['Company', 'Date', 'Observations', 'orig_perm_id']], label0[['water']], on='orig_perm_id', how='left').drop_duplicates()
df = pd.merge(df, label1[['trees']], on='orig_perm_id', how='left')
df = pd.merge(df, label2[['grass']], on='orig_perm_id', how='left')
df = pd.merge(df, label3[['flooded_vegetation']], on='orig_perm_id', how='left')
df = pd.merge(df, label4[['crops']], on='orig_perm_id', how='left')
df = pd.merge(df, label5[['shrub_and_scrub']], on='orig_perm_id', how='left')
df = pd.merge(df, label6[['built']], on='orig_perm_id', how='left')
df = pd.merge(df, label7[['bare']], on='orig_perm_id', how='left')
df = pd.merge(df, label8[['snow_and_ice']], on='orig_perm_id', how='left')
df.fillna(0, inplace=True)
df

In [28]:
len(df)

31567

In [29]:
len(pd.merge(df, mines,  how='inner', left_on=['orig_perm_id'], right_on = ['permit_id']))

22560

In [42]:
df = pd.merge(df, mines,  how='inner', left_on=['orig_perm_id'], right_on = ['permit_id'])

In [43]:
df['water'] = df['water']/df['Observations']
df['trees'] = df['trees']/df['Observations']
df['grass'] = df['grass']/df['Observations']
df['flooded_vegetation'] = df['flooded_vegetation']/df['Observations']
df['shrub_and_scrub'] = df['shrub_and_scrub']/df['Observations']
df['bare'] = df['bare']/df['Observations']
df['built'] = df['built']/df['Observations']
df


,Company,Date,Observations,orig_perm_id,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,...,permit_approval_date,edit_date,area_mine,contour,mountaintop,steep_slope,highwall,auger,contact,information_link
0,RIDNER COAL CO INC,2023-07-01,8,9180010,0.0,1.00,0.00,0.0,0.0,0.0,...,1984-06-12,2000-07-26,0.0,1.0,0.0,0.0,0.0,0.0,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...
1,RIDNER COAL CO INC,2021-07-01,8,9180010,0.0,1.00,0.00,0.0,0.0,0.0,...,1984-06-12,2000-07-26,0.0,1.0,0.0,0.0,0.0,0.0,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...
2,RIDNER COAL CO INC,2019-07-01,8,9180010,0.0,1.00,0.00,0.0,0.0,0.0,...,1984-06-12,2000-07-26,0.0,1.0,0.0,0.0,0.0,0.0,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...
3,HERBERT WELLS,2023-07-01,8,0320080,0.0,0.75,0.25,0.0,0.0,0.0,...,1899-12-30,1999-11-30,0.0,0.0,0.0,0.0,0.0,0.0,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...
4,HERBERT WELLS,2021-07-01,8,0320080,0.0,0.75,0.25,0.0,0.0,0.0,...,1899-12-30,1999-11-30,0.0,0.0,0.0,0.0,0.0,0.0,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22555,"MURRAY AMERICAN ENERGY, INC.",2017-07-01,1,U201111,0.0,0.00,1.00,0.0,0.0,0.0,...,2013-09-09,NaT,0.0,0.0,0.0,0.0,NaN,0.0,2,https://dep.wv.gov/dmr/Pages/default.aspx
22556,MINGO LOGAN COAL LLC,2017-07-01,2,U501997,0.0,1.00,0.00,0.0,0.0,0.0,...,1997-08-19,NaT,0.0,0.0,0.0,0.0,NaN,0.0,2,https://dep.wv.gov/dmr/Pages/default.aspx
22557,"FARMERS CONSTRUCTION COMPANY, INC.",2017-07-01,1,S005080,0.0,0.00,0.00,0.0,0.0,0.0,...,1980-06-02,NaT,1.0,1.0,0.0,0.0,NaN,0.0,2,https://dep.wv.gov/dmr/Pages/default.aspx
22558,"MOUNTAIN EMPIRE LEASING, INC.",2017-07-01,2,S400792,0.0,1.00,0.00,0.0,0.0,0.0,...,1992-08-14,NaT,1.0,1.0,1.0,0.0,NaN,1.0,2,https://dep.wv.gov/dmr/Pages/default.aspx


In [44]:
df

,Company,Date,Observations,orig_perm_id,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,...,permit_approval_date,edit_date,area_mine,contour,mountaintop,steep_slope,highwall,auger,contact,information_link
0,RIDNER COAL CO INC,2023-07-01,8,9180010,0.0,1.00,0.00,0.0,0.0,0.0,...,1984-06-12,2000-07-26,0.0,1.0,0.0,0.0,0.0,0.0,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...
1,RIDNER COAL CO INC,2021-07-01,8,9180010,0.0,1.00,0.00,0.0,0.0,0.0,...,1984-06-12,2000-07-26,0.0,1.0,0.0,0.0,0.0,0.0,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...
2,RIDNER COAL CO INC,2019-07-01,8,9180010,0.0,1.00,0.00,0.0,0.0,0.0,...,1984-06-12,2000-07-26,0.0,1.0,0.0,0.0,0.0,0.0,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...
3,HERBERT WELLS,2023-07-01,8,0320080,0.0,0.75,0.25,0.0,0.0,0.0,...,1899-12-30,1999-11-30,0.0,0.0,0.0,0.0,0.0,0.0,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...
4,HERBERT WELLS,2021-07-01,8,0320080,0.0,0.75,0.25,0.0,0.0,0.0,...,1899-12-30,1999-11-30,0.0,0.0,0.0,0.0,0.0,0.0,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22555,"MURRAY AMERICAN ENERGY, INC.",2017-07-01,1,U201111,0.0,0.00,1.00,0.0,0.0,0.0,...,2013-09-09,NaT,0.0,0.0,0.0,0.0,NaN,0.0,2,https://dep.wv.gov/dmr/Pages/default.aspx
22556,MINGO LOGAN COAL LLC,2017-07-01,2,U501997,0.0,1.00,0.00,0.0,0.0,0.0,...,1997-08-19,NaT,0.0,0.0,0.0,0.0,NaN,0.0,2,https://dep.wv.gov/dmr/Pages/default.aspx
22557,"FARMERS CONSTRUCTION COMPANY, INC.",2017-07-01,1,S005080,0.0,0.00,0.00,0.0,0.0,0.0,...,1980-06-02,NaT,1.0,1.0,0.0,0.0,NaN,0.0,2,https://dep.wv.gov/dmr/Pages/default.aspx
22558,"MOUNTAIN EMPIRE LEASING, INC.",2017-07-01,2,S400792,0.0,1.00,0.00,0.0,0.0,0.0,...,1992-08-14,NaT,1.0,1.0,1.0,0.0,NaN,1.0,2,https://dep.wv.gov/dmr/Pages/default.aspx


In [49]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=xluciabV3jO5ISkIiZBR-yb-SaKXaOEcRZRCOVSBgR8&tc=H6JlCT5nezfScabu92lvHpT8hO6hIPFClMPR-IRk4SQ&cc=SN8gJzhsoEL8O8JT0LP2Bk-uCtB3kKmoui1vo8_NS0M

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BUc86kCeXnwqUe0uT8yH-3Aoug9T_8jsCYJ2XCMukKbC9KJu4JG5oM

Successfully saved authorization token.


In [108]:
dataquery = df[['longitude','latitude','start_date','end_date', 'orig_perm_id']].drop_duplicates()
dataquery['end_date'] = dataquery['end_date'].astype(str)
dataquery['start_date'] = dataquery['start_date'].astype(str)

In [ ]:
ee.ImageCollection("IDAHO_EPSCOR/GRIDMET")\
        .filterBounds(point) \
        .filterDate(start_date, end_date) \
        .first() \
        .normalizedDifference(['B3', 'B8']) \

In [122]:
temperature.getInfo())

SyntaxError: ignored

In [133]:
temperature_metrics:properties:

SyntaxError: ignored

In [145]:
check = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET")\
        .filterBounds(point) \
        .filterDate(start_date, end_date) \
        .first()

In [149]:
test = check.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=30
    ).get('pr', 'tmmx')
test

check.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=30
    ).get('tmmx')


In [ ]:
temperature_metrics = []
water_quality_metrics = []
rain_metrics = []
i=0

while i < len(dataquery):
    # Access the geometry of the feature
    data = dataquery[i:(i+1)]
    longitude = data['longitude'].values
    latitude = data['latitude'].values

    start_date = data['start_date'].values[0]
    end_date = data['end_date'].values[0]

    # Create an Earth Engine point geometry
    point = ee.Geometry.Point(longitude[0], latitude[0])

    # Retrieve water quality metrics using satellite imagery
    water_quality = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(point) \
        .filterDate(start_date, end_date) \
        .first() \
        .normalizedDifference(['B3', 'B8']) \
        .rename('water_quality')

    temperature = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET")\
        .filterBounds(point) \
        .filterDate(start_date, end_date) \
        .first() \

    rain = temperature.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=30
        ).get('pr')

    temp = temperature.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=30
        ).get('tmmx')

    # Calculate water quality statistics within a region of interest (e.g., buffer around the point)
    water_quality_stats = water_quality.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point.buffer(500),  # Adjust buffer size as needed
        scale=10
    ).get('water_quality')

    print(temp.getInfo())
    print(rain.getInfo())
    print(water_quality_stats.getInfo())

    # Append the water quality metric to the list
    water_quality_metrics.append(water_quality_stats.getInfo())
    temperature_metrics.append(temp.getInfo())
    rain_metrics.append(rain.getInfo())
    i+=1


298.5
3.5999999046325684
0.03468842620517674
298.5
3.5999999046325684
0.013022294955906831
298.5
3.5999999046325684
0.018867105744238166
298.5
3.5999999046325684
0.0250793219234676


In [ ]:
# Add latitude, longitude, elevation, and water quality columns
dataquery['Water Quality'] = water_quality_metrics

# Print the updated DataFrame
print(dataquery)